# Welcome to the Lab 🥼🧪

Create techincal charts to analyze pricing behavior against demand. 

In [ ]:
import os
import sys
import json

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido numpy
    !git clone https://github.com/ParclLabs/parcllabs-cookbook.git
    sys.path.append('/content/parcllabs-cookbook/examples/')
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')
    cur_dir = os.getcwd()
    chart_dir = os.path.join(cur_dir, '..')
    sys.path.append(chart_dir)

In [ ]:
import parcllabs
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from parcllabs import ParclLabsClient

api_key = os.getenv('PARCL_LABS_API_KEY')
print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# import technical charts
from charting.pricefeed_utils import TimeSeriesAnalysis
from charting.name_formats import format_names, create_ticker, format_cs_10_names
from charting.technical_charts import technical_pricefeed_charts, calculate_stats


In [ ]:
# Initialize the Parcl Labs client
client = ParclLabsClient(api_key)

In [ ]:
# set nb config
pf_options = {
    'rental': 'rental_price_feed',
    'pricefeed': 'price_feed'
}

PF_TYPE = pf_options['pricefeed']

In [ ]:
# Get top 100 CBSAs by population
markets = client.search_markets.retrieve(
    as_dataframe=True,
    sort_by='PARCL_EXCHANGE_MARKET',
    sort_order='DESC',
    params={
        'limit': 15
    }
)
markets['clean_name'] = markets['name'].apply(format_cs_10_names)
markets['ticker'] = markets['clean_name'].apply(create_ticker)
markets

In [ ]:
# lets retrieve data back to 2011 for these price feeds
START_DATE = '2019-01-01'
feeds = client.price_feed.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
    auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

In [ ]:
# get supply and demand
demand = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
)

In [ ]:
LAST_CS_UPDATE = '2024-03-31'
TICKER_MESSAGE = "<b>{ticker}</b> {name} Parcl Exchange"

for pid in markets['parcl_id']:

    mf = feeds[feeds['parcl_id'] == pid]
    df = demand[demand['parcl_id'] == pid]
    supply = demand[demand['parcl_id'] == pid]

    # Ensure the DataFrame is sorted by date and calculate daily return
    mf = mf.sort_values('date')
    last_pricefeed_value = mf['price_feed'].iloc[-1]
    last_pricefeed_date_str = mf['date'].iloc[-1]
    last_pricefeed_date = pd.to_datetime(last_pricefeed_date_str).strftime('%-d %B')
    last_pf_date = pd.to_datetime(last_pricefeed_date_str).strftime('%-d-%B-%Y')
    mf['date'] = pd.to_datetime(mf['date'])
    mf['daily_return'] = mf['price_feed'].pct_change()
    mf = mf.dropna()  # Remove rows with NaN values resulting from pct_change calculation

    mf['month'] = mf['date'].dt.to_period('M')
    monthly_return = mf.groupby('month')['price_feed'].agg(lambda x: x.iloc[-1] / x.iloc[0] - 1).reset_index(name='volColor')
    df = df.sort_values('date')
    supply = supply.sort_values('date')
    last_sales_value = df['sales'].iloc[-1]
    last_sales_month = pd.to_datetime(df['date'].iloc[-1]).strftime('%b')
    last_supply_value = supply['new_listings_for_sale'].iloc[-1]
    last_supply_month = supply['date'].iloc[-1]
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.to_period('M')
    supply['date'] = pd.to_datetime(supply['date'])
    supply['month'] = supply['date'].dt.to_period('M')
    df = df.merge(monthly_return, on='month')
    supply = supply.merge(monthly_return, on='month')

    # Add volColor column directly to data_secondary
    df['volColor'] = np.where(df['volColor'] >= 0, 'green', 'red')  # Modify this condition as per your actual data logic
    supply['volColor'] = np.where(supply['volColor'] >= 0, 'green', 'red')  # Modify this condition as per your actual data logic
    ticker = markets[markets['parcl_id'] == pid]['ticker'].values[0]
    name = markets[markets['parcl_id'] == pid]['clean_name'].values[0]

    high_52w, low_52w, last_value, yoy_change_delta, yoy_change_pct, yoy_sign = calculate_stats(mf)
    msg = f"<b>52w High</b> {high_52w:.2f} <b>52w Low</b> {low_52w:.2f} <b>Last</b> {last_value:.2f} <b>YoY Change</b> {round(yoy_change_delta, 2)} ({yoy_change_pct*100:.02f}%)"
    # msg = f"52w High ${high_52w:.2f} 52w Low ${low_52w:.2f} Last ${last_value:.2f} YoY Change {round(yoy_change_delta, 2)} ({yoy_change_pct:.2f}%)"
    volume_msg = f'Sales Volume (Monthly) {last_sales_value:,} ({last_sales_month})'
    pricefeed_msg = f'{ticker} (Daily) ${last_pricefeed_value} ({last_pricefeed_date})'
    ticker_msg = TICKER_MESSAGE.format(ticker=ticker, name=name)

    pf_ts_analysis = TimeSeriesAnalysis(mf, 'date', 'price_feed', freq='D')
    pf_rate_of_change_stats = pf_ts_analysis.calculate_changes(change_since_date=LAST_CS_UPDATE)

    demand_ts_analysis = TimeSeriesAnalysis(df, 'date', 'sales', freq='M')
    demand_rate_of_change_stats = demand_ts_analysis.calculate_changes(change_since_date=LAST_CS_UPDATE)

    rate_of_change_analysis = {
        'name': name,
        'parcl_id': pid,
        'pricefeed': pf_rate_of_change_stats,
        'demand': demand_rate_of_change_stats
    }

    # dump output
    with open(f'../../graphics/pf_vs_volume_charts/parcl_exchange/2024-06-17/{name}_pf_vs_volume.json', 'w') as f:
        json.dump(rate_of_change_analysis, f)

    technical_pricefeed_charts(
        data_main=mf,
        data_secondary=df,
        value_name_main='price_feed',
        value_name_secondary='sales',
        ticker_msg=ticker_msg,  
        msg=msg,
        volume_msg=volume_msg,
        pricefeed_msg=pricefeed_msg,
        last_pf_date=last_pf_date,
        save_path=f'../../graphics/pf_vs_volume_charts/parcl_exchange/2024-06-17/{name}_pf_vs_volume.png'
    )